<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#merge()详解" data-toc-modified-id="merge()详解-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>merge()详解</a></span><ul class="toc-item"><li><span><a href="#1.-函数说明" data-toc-modified-id="1.-函数说明-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1. 函数说明</a></span></li><li><span><a href="#2.on-用法" data-toc-modified-id="2.on-用法-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>2.on 用法</a></span></li><li><span><a href="#3.-how-用法" data-toc-modified-id="3.-how-用法-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>3. how 用法</a></span></li><li><span><a href="#对齐的列存在重复值" data-toc-modified-id="对齐的列存在重复值-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>对齐的列存在重复值</a></span></li></ul></li><li><span><a href="#concat()详解" data-toc-modified-id="concat()详解-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>concat()详解</a></span></li></ul></div>

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame

## merge()详解
### 1. 函数说明

pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort = False)

**concat函数本质上是在所有索引上同时进行对齐合并，而如果想在任意列上对齐合并，则需要merge函数，其在sql应用很多。**

- left,right： 两个要对齐合并的DataFrame；
- how： 先做笛卡尔积操作，然后按照要求，保留需要的，缺失的数据填充NaN；
  - left: 以左DataFrame为基准，即左侧DataFrame的数据全部保留（不代表完全一致、可能会存在复制），保持原序
  - right: 以右DataFrame为基准，保持原序
  - inner: 交，保留左右DataFrame在on上完全一致的行，保持左DataFrame顺序
  - outer: 并，按照字典顺序重新排序
- on：列索引或列索引列表，如果要在DataFrame相同的列索引做对齐，用这个参数；
- left_on, right_on, left_index, right_index：
  - on对应普通的列索引或列索引列表，对齐不同列名的DataFrame，用这俩参数；
  - index对应要使用的index，建议不要使用这俩参数，因为可以用concat方法代替。
- sort: True or False，是否按字典序重新排序。

In [6]:
df1 = DataFrame([[1,2],[3,4]], index = ['a','b'],columns = ['A','B'])
df2 = DataFrame([[1,3],[4,8]], index = ['b','d'],columns = ['B','C'])
df1
df2

,A,B
a,1,2
b,3,4


,B,C
b,1,3
d,4,8


In [7]:
pd.merge(left = df1, right = df2, left_on='A', right_on='B')

,A,B_x,B_y,C
0,1,2,1,3


In [ ]:
如果单纯的按照index对齐，不如用concat方法，所以一般不建议使用left_index, right_index 。


In [22]:
df3 = DataFrame([[1,2],[3,4]], index = ['a','b'],columns = ['A','B'])
df4 = DataFrame([[5,6],[7,8]], index = ['c','b'],columns = ['C','B'])
df3
df4
pd.merge(left = df3, right = df4,left_on='A', right_on='B')

,A,B
a,1,2
b,3,4


,C,B
c,5,6
b,7,8


,A,B_x,C,B_y


In [16]:
df3=pd.DataFrame({'l_key':['b','b','a','a','b','a','c'],'data1':range(7)})
df3
df4=pd.DataFrame({'r_key':['a','b','d'],'data2':range(3)})
df4
pd.merge(df3,df4,left_on='l_key',right_on='r_key')


,l_key,data1
0,b,0
1,b,1
2,a,2
3,a,3
4,b,4
5,a,5
6,c,6


,r_key,data2
0,a,0
1,b,1
2,d,2


,l_key,data1,r_key,data2
0,b,0,b,1
1,b,1,b,1
2,b,4,b,1
3,a,2,a,0
4,a,3,a,0
5,a,5,a,0


In [23]:
# 小区别是concat对重复列没有重命名，但是重名的情况不多，而且重名了说明之前设计就不大合理。
pd.concat([df1,df2], join = 'inner',axis =1)  

,A,B,B,C
b,3,4,1,3


### 2.on 用法
设置 how = 'inner'

In [24]:
#对于'B'列：df1的'b'行、df2的'd'行，是相同的，其他都不同。 
pd.merge(left = df1, right = df2, how = 'inner' , on =['B']) 

,A,B,C
0,3,4,8


In [25]:
# df1的'A'列'b'行，df2的'C'列'd'行是相同的，其他都不同。
# 其他列如果同名会进行重命名。
pd.merge(left = df1, right = df2, how = 'inner',left_on = ['A'] ,right_on = ['C'])


,A,B_x,B_y,C
0,3,4,1,3


### 3. how 用法

In [26]:
# 保持左侧DataFrame不变，用右侧来跟它对齐，对不上的填NaN。
pd.merge(left = df1, right = df2, how = 'left', on = ['B'] )


,A,B,C
0,1,2,NaN
1,3,4,8.0


In [27]:
# 保持右侧DataFrame不变，用右侧来跟它对齐，对不上的填NaN。
pd.merge(left = df1, right = df2, how = 'right', on = ['B'] )


,A,B,C
0,3.0,4,8
1,NaN,1,3


### 对齐的列存在重复值

重复的也没关系，操作逻辑是一致的，完全可以假想不存在重复。


In [ ]:
df1.loc['a','B'] =4  #改成重复
df1

In [28]:
### 保持右侧的列都在，如果左侧对齐的列存在重复值，那么对齐上后也存在重复。
pd.merge(left = df1, right = df2, how = 'right', on = ['B'] )


,A,B,C
0,3.0,4,8
1,NaN,1,3


## concat()详解
函数说明

pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=None, copy=True)


In [33]:
s1 = Series([1, 2, 3], index=['a', 'b', 'c'])
s2 = Series([4, 5, 6], index=['d', 'e', 'f'])
s3 = Series([7, 8, 9], index=['g', 'h', 'i'])
pd.concat([s1, s2, s3])



a    1
b    2
c    3
d    4
e    5
f    6
g    7
h    8
i    9
dtype: int64

In [34]:
pd.concat([s1, s2, s3], axis=1, sort=False)

,0,1,2
a,1.0,NaN,NaN
b,2.0,NaN,NaN
c,3.0,NaN,NaN
d,NaN,4.0,NaN
e,NaN,5.0,NaN
f,NaN,6.0,NaN
g,NaN,NaN,7.0
h,NaN,NaN,8.0
i,NaN,NaN,9.0


In [ ]:



a    1
b    2
c    3
d    4
e    5
f    6
g    7
h    8
i    9
dtype: int64
